In [ ]:
import random
from random import choice
import torch

In [ ]:
class pycoder_parameters:

    Path = 'gen1_33'
    ''' Core Fuzzing Parameters '''
    NUM_FUZZ_PER_API= 100001
    NUM_TEST_FUZZ = 2
    UNIT_TEST = False
    COMPOSITE = True

    ''' Fuzzing Detailed Parameters '''
    MAX_TENSOR_DIMENSIONS = 3 #how many rows, columns, etc.
    MIN_VAL_PER_DIMENSION = 1 # e.g., min number of rows, columns, etc. 
    MAX_VAL_PER_DIMENSION = 5 # e.g., max number of rows, columns, etc. 

    #So far limiting to integers
    MIN_TENSOR_VALUE = 0
    MAX_TENSOR_VALUE = 20
    

    ''' Embedding Parameters '''
    EMBEDDING_NOISE_LEVEL = 0 #0 noise by default
    EMBEDDING_SIZE = 150
    SHAPE_EMBEDDING_SIZE = 6


    data_type = 'integer'
    model_type = 'Composite_' if COMPOSITE is  True else 'Single_'
    file_name = str(model_type) + str(NUM_FUZZ_PER_API) + '_' + data_type
    fuzzing   = file_name + '.pt'
    embedding = file_name + '.embedding' + '.pt',
    classification = file_name + '.model_result' + '.pt' 

    def setNoiseLevel(self, noise):
        self.EMBEDDING_NOISE_LEVEL = noise
        self.embedding = self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '.pt'

    def getEmbeddingFile(self):
        return(self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '.pt')

    def getVisulizationFile(self):
        return(self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '_' +  'tSNE.pt')

In [ ]:
f = pycoder_parameters()

dimension_choice = [j for j in range(1,f.MAX_TENSOR_DIMENSIONS+1)]

UNIT_TEST = f.UNIT_TEST
NUM_FUZZ_PER_API= f.NUM_FUZZ_PER_API
NUM_TEST_FUZZ = f.NUM_TEST_FUZZ
UNIT_TEST = f.UNIT_TEST
    
MAX_TENSOR_DIMENSIONS = f.MAX_TENSOR_DIMENSIONS
MIN_VAL_PER_DIMENSION = f.MIN_VAL_PER_DIMENSION
MAX_VAL_PER_DIMENSION = f.MAX_VAL_PER_DIMENSION 

#So far limiting to integers
MIN_TENSOR_VALUE = f.MIN_TENSOR_VALUE
MAX_TENSOR_VALUE = f.MAX_TENSOR_VALUE
MAX_ELEMS = f.EMBEDDING_SIZE

''' Declaring them here to avoid compilation error, but they are intialized later from the core fuzzing function'''
api_dict = {}
api_list = []

In [ ]:
def get_tensor(random_min, random_max, dim_list):
    
    shape = tuple(dim_list)
    tensor = torch.randint(random_min,random_max, shape)
    
    return(tensor)

def get_broadcastable_size(size):
    
    broadcastable_size = []
    for i, d in enumerate(reversed(size)):
        if random.random() < 0.75:
            broadcastable_size.insert(0, random.choice([1, d]))
            if len(broadcastable_size) == MAX_TENSOR_DIMENSIONS:
                break
        else:
            if d == 1 and random.random() < 0.5:
                broadcastable_size.insert(0, random.randint(1, MAX_VAL_PER_DIMENSION))
            else:
                break
    if len(broadcastable_size) < MAX_TENSOR_DIMENSIONS and random.random() < 0.3:
        broadcastable_size.insert(0, 1)
    if len(broadcastable_size) == 0:
        broadcastable_size = get_broadcastable_size(size)
    return broadcastable_size
   

def get_broadcast_tensor(original_tensor, min_val = MIN_TENSOR_VALUE, max_val = MAX_TENSOR_VALUE, dim_list=None):
    if dim_list is None:
        dim_list = list(original_tensor.shape)

    dim_list1 = get_broadcastable_size(dim_list)
    broadcast_tensor = get_tensor(min_val, max_val, dim_list=dim_list1)
    return(broadcast_tensor)


In [ ]:
def swap_tensor(tensor1, tensor2):
    return((tensor2,tensor1))

if UNIT_TEST is True:
    t1 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,[1])
    t2 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,[1,1])
    print(t1, t2)
    t1, t2 = swap_tensor(t1,t2)
    print(t1, t2)


In [ ]:
def fuzz_expand(input_tensor_list=None):
   
    api_name0 = 'expand'
    log_input_output = []

    if input_tensor_list is None:
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(choice(dimension_choice)) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])


    dimension = list(input_tensor.shape)
    expanded_dimension = dimension.copy()
    expanded = False
    for i, d in enumerate(dimension):
        if d == 1:
            expanded_dimension[i] = random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)
            expanded = True
    
    if expanded:
        output_tensor = input_tensor.expand(*expanded_dimension)
        log_input_output.append([api_name0,output_tensor,[input_tensor]])
        
    else:
        expanded_dimension = [random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)] + expanded_dimension
        output_tensor = input_tensor.expand(*expanded_dimension)
        log_input_output.append([api_name0,output_tensor,[input_tensor]])
    
    return(log_input_output)


'''unit_test'''
def test_fuzz_expand(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_expand()#(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_expand" , t)


if UNIT_TEST is True:     
    test_fuzz_expand()

In [ ]:
'''
    Returns a tensor that is a transposed version of input. 
    The given dimensions dim0 and dim1 are swapped.
'''
def fuzz_transpose(input_tensor_list=None):

    api_name0 = 'transpose'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension+1) ]  
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list) 
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    
    if len(list(input_tensor.shape)) == 1:
        return([])

    dimension = (list(input_tensor.size()))
    dims = random.sample(range(0, len(dimension)), 2)
    
    output_tensor = torch.transpose(input_tensor, dims[0], dims[1])

    log_input_output.append([api_name0,output_tensor,[input_tensor]])
    
    return(log_input_output)


'''unit_test'''
def test_fuzz_transpose(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_transpose()
        print("fuzz_transpose" , t)


if UNIT_TEST is True:
    test_fuzz_transpose()

In [ ]:
def fuzz_lt(input_tensor_list=None):

    api_name0 = 'lt'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    
    input_tensor1 = get_tensor(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor.shape))

    dimension = len(list(input_tensor1.size()))
     
    if random.random() < 0.5:
        if random.random() < 0.5:
            input_tensor, input_tensor1 = swap_tensor(input_tensor, input_tensor1)
        output_tensor = torch.lt(input_tensor1, input_tensor) 
        log_input_output.append([api_name0,output_tensor,[input_tensor, input_tensor1]])
    else:
        scalar_value = random.randint(torch.min(input_tensor).item(), torch.max(input_tensor).item()) 
        output_tensor = torch.lt(input_tensor, scalar_value) 
        t = torch.tensor(scalar_value)
        log_input_output.append([api_name0,output_tensor,[input_tensor, t]])

    return(log_input_output)

'''unit_test'''
def test_fuzz_lt(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_lt() #MIN_TENSOR_VALUE,MIN_TENSOR_VALUE+5,choice(dimension_choice))
        print("fuzz_lt", log_input_output)

if UNIT_TEST is True:    
    test_fuzz_lt()

In [ ]:
'''
 Return a tensor of elements selected from either x or y, depending on condition.
'''

def fuzz_where(input_tensor_list=None):

    api_name0 = 'where'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor0 = torch.rand(tuple(dim_list)) < 0.50
        input_tensor1 = get_broadcast_tensor(input_tensor0) #MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor0.shape))#.float()
        input_tensor2 = get_broadcast_tensor(input_tensor0) #MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor0.shape))

    else:
        input_tensor = input_tensor_list[0]

        if input_tensor.dtype == torch.bool:
            input_tensor0 = input_tensor
            input_tensor1 = get_broadcast_tensor(input_tensor0) #MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor0.shape))#.float()
            input_tensor2 = get_broadcast_tensor(input_tensor0) #MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor0.shape))
        else:
            input_tensor0 = torch.rand(tuple(input_tensor.shape)) < 0.75
            if random.random() < 0.5:
                input_tensor1 = input_tensor
                if random.random() < 0.5:
                    input_tensor2 = get_broadcast_tensor(input_tensor) #MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor.shape))
                else:
                    input_tensor2 = torch.tensor(random.randint(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE))
            else:
                input_tensor2 = input_tensor
                if random.random() < 0.5:
                    input_tensor1 = get_broadcast_tensor(input_tensor) #MIN_TENSOR_VALUE, MAX_TENSOR_VALUE,list(input_tensor.shape))
                else:
                    input_tensor1 = torch.tensor(random.randint(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE))
    try:
        output_tensor = torch.where(input_tensor0, input_tensor1, input_tensor2) #some condition
        log_input_output.append([api_name0,output_tensor,[input_tensor0, input_tensor1, input_tensor2]])
    except:
        pass
    
    return(log_input_output)

'''unit_test'''
def test_fuzz_where(): 
    for i in range(1,NUM_TEST_FUZZ):#NUM_FUZZ_PER_API): #number of call
        log_input_output = fuzz_where() #[torch.tensor([False,  True])])
        print("fuzz_where", log_input_output)

if UNIT_TEST is True:     
    test_fuzz_where()

In [ ]:
def fuzz_minimum(input_tensor_list=None):
    
    api_name0 = 'minimum'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor  = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])


    if random.random() < 0.10:
        scalar = random.randint(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE)
        input_tensor1 = torch.tensor(scalar)
        output_tensor = torch.minimum(input_tensor, input_tensor1) 
        log_input_output.append([api_name0,output_tensor,[input_tensor, input_tensor1]])
    else:
        input_tensor1 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,list(input_tensor.shape))
        if random.random() < 0.5:
            input_tensor, input_tensor1 = swap_tensor(input_tensor, input_tensor1)
        output_tensor = torch.minimum(input_tensor, input_tensor1) 
        log_input_output.append([api_name0,output_tensor,[input_tensor, input_tensor1]])
        
    return(log_input_output)

'''unit_test'''
def test_fuzz_minimum(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_minimum() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_minimum", log_input_output)
        
if UNIT_TEST is True:
    test_fuzz_minimum()

In [ ]:
def fuzz_tensordot(input_tensor_list=None):

    api_name0 = 'tensordot'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        size = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension+1) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),size)        
    else:
        input_tensor  = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
        size = list(input_tensor.shape)
        

    reducing_dim = random.randint(1, len(size))
    in2_dim = random.randint(reducing_dim, MAX_TENSOR_DIMENSIONS+1)
    size2 = size[-reducing_dim:] + [random.randint(1, MAX_VAL_PER_DIMENSION) for _ in range(in2_dim-reducing_dim)] 

    in2 = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),size2)
    output_tensor = torch.tensordot(input_tensor, in2, dims=reducing_dim)

    log_input_output.append([api_name0,output_tensor,[input_tensor, in2]])  

    return(log_input_output)

'''unit_test'''
def test_fuzz_tensordot(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_tensordot()#MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_tensordot" , t)

if UNIT_TEST is True:
    test_fuzz_tensordot()

In [ ]:
'''
 Returns a new tensor with a dimension of size one inserted at the specified position.
'''
def fuzz_unsqueeze(input_tensor_list=None):

    api_name0 = 'unsqueeze'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE, dim_list)
    else: 
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    
    dimension = len(list(input_tensor.size()))
    unsqueeze_dim = random.randint(0,dimension)
    output_tensor = torch.unsqueeze(input_tensor, unsqueeze_dim)

    log_input_output.append([api_name0,output_tensor,[input_tensor]])

    return(log_input_output)

'''unit_test'''
def test_fuzz_unsqueeze(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_unsqueeze() #(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_unsqueeze", t)


if UNIT_TEST is True:
    test_fuzz_unsqueeze()

In [ ]:
'''
    Returns a tensor with all the dimensions of input of size 1 removed.
'''

def fuzz_squeeze(input_tensor_list=None):

    api_name = 'squeeze'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE, dim_list)
    else: 
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
   
    dimension = len(list(input_tensor.size()))
    dim = random.randint(0, dimension)

    if dim == len(list(input_tensor.shape)):
        output_tensor = torch.squeeze(input_tensor)
    else:
        output_tensor = torch.squeeze(input_tensor, dim=dim)
    
    
    log_input_output.append([api_name,output_tensor,[input_tensor]])
    
    return(log_input_output)


'''unit_test'''
def test_fuzz_squeeze(): 
    for i in range(1,NUM_FUZZ_PER_API): #number of call
        t=fuzz_squeeze()#MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_squeeze", t)

if UNIT_TEST is True:
    test_fuzz_squeeze()

In [ ]:
def fuzz_eq(input_tensor_list=None):

    api_name0 = "eq"

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE, dim_list)
    else: 
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    
    input_tensor1 = get_broadcast_tensor(input_tensor, MIN_TENSOR_VALUE, MIN_TENSOR_VALUE+5)
    
    int_flat = input_tensor.flatten()
    int_flat1 = input_tensor1.flatten()
    scalar = 0
    for replace in range(2):
        if(int_flat.shape[0] != 1 or int_flat1.shape[0] != 1):
            
            i = random.randint(0,int_flat.shape[0]-1)
            j = random.randint(0,int_flat1.shape[0]-1)
            k = min(i,j)
            int_flat1[k] = int_flat[k]

            input_tensor1 = int_flat1.reshape(tuple(input_tensor1.shape))

    if random.random() < 0.5:
        tmp_tensor = input_tensor
        input_tensor = input_tensor1
        input_tensor1 = tmp_tensor

    if random.random() < 0.5:
        output_tensor = torch.eq(input_tensor, input_tensor1) 
    else:
        scalar = random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)
        input_tensor1 = torch.tensor(scalar)
        output_tensor = torch.eq(input_tensor, input_tensor1) 
    log_input_output.append([api_name0, output_tensor, [input_tensor, input_tensor1]])
    
    return(log_input_output)


'''unit_test'''
def test_fuzz_eq(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_eq() #(MIN_TENSOR_VALUE,MIN_TENSOR_VALUE+5,choice(dimension_choice))
        print("fuzz_eq", log_input_output)
        
if UNIT_TEST is True:
    test_fuzz_eq()

In [ ]:
def fuzz_gt(input_tensor_list=None):

    api_name0 = 'gt'
    
    log_input_output = []

    tensor_dimension=choice(dimension_choice)
    dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]

    if input_tensor_list is None:
        input_tensor = get_tensor(MIN_TENSOR_VALUE, MAX_TENSOR_VALUE, dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
        
    if random.random() < 0.5:
        scalar_value = random.randint(0, MAX_TENSOR_VALUE)
        output_tensor = torch.gt(input_tensor, scalar_value) 
        log_input_output.append([api_name0, output_tensor,[input_tensor, torch.tensor(scalar_value)]])
    else:
        input_tensor1 = get_broadcast_tensor(input_tensor, MIN_TENSOR_VALUE, MAX_TENSOR_VALUE)
        output_tensor = torch.gt(input_tensor, input_tensor1) 
        log_input_output.append([api_name0, output_tensor,[input_tensor, input_tensor1]])
    
    return(log_input_output)

'''unit_test'''
def test_fuzz_gt(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_gt() #MIN_TENSOR_VALUE,MIN_TENSOR_VALUE+5,choice(dimension_choice))
        print("fuzz_gt", log_input_output)

if UNIT_TEST is True:       
    test_fuzz_gt()

In [ ]:
'''
  Returns a new 1-D tensor which indexes the input tensor according to the boolean mask which is a BoolTensor.
'''
def fuzz_masked_select(input_tensor_list=None):
   
    api_name0 = 'masked_select'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]

    if input_tensor.dtype == torch.bool:
        input_tensor1 = input_tensor
        input_tensor0 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,list(input_tensor.shape))
    else:
        input_tensor0 = input_tensor
        temp_tensor = get_broadcast_tensor(input_tensor, MIN_TENSOR_VALUE,MAX_TENSOR_VALUE)
        input_tensor1 = temp_tensor.gt(random.randint(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE))


    output_tensor = torch.masked_select(input_tensor0, input_tensor1)
    log_input_output.append([api_name0,output_tensor,[input_tensor0, input_tensor1]])
   
    return(log_input_output)

'''unit_test'''
def test_fuzz_masked_select(): 
    for i in range(1, NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_masked_select()#MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_masked_select", log_input_output)

if UNIT_TEST is True:
    test_fuzz_masked_select()

In [ ]:
'''
 Count the frequency of each value in an array of non-negative ints.
'''
def fuzz_bincount(input_tensor_list=None):  
    
    api_name0 = 'bincount'
    
    log_input_output = []
    tensor_dimension=1 #constraint: this is always for 1-D array
    dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
   
    if input_tensor_list is None:
        input_tensor = get_tensor(0,max(dim_list),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        
        if input_tensor.dtype == torch.bool:
            return([-1])

        if len(list(input_tensor.shape)) != 1:
            return(log_input_output)

    api_name = api_name0
    output_tensor = torch.bincount(input_tensor)
    log_input_output.append([api_name0,output_tensor,[input_tensor]])
    return(log_input_output)


'''unit_test'''

def test_fuzz_bincount():    
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_bincount()
        print("fuzz_bincount",t)

if UNIT_TEST is True:
    test_fuzz_bincount()

In [ ]:
def fuzz_matmul(input_tensor_list=None):

   
    api_name0 = 'matmul'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    

    if input_tensor_list and len(input_tensor_list) >= 2:

        input_tensor1 = input_tensor_list[1]
        output_tensor = torch.matmul(input_tensor, input_tensor1) 
        log_input_output.append([api_name0,output_tensor,[input_tensor, input_tensor1]])
    
    else:
        min_val = MIN_TENSOR_VALUE
        max_val = int(MAX_TENSOR_VALUE/2)

        dim_list = list(input_tensor.shape)
               

        if len(dim_list) == 1:
            if random.random() < 0.5:
                input_tensor1 = get_tensor(min_val,max_val,dim_list) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])
            else:
                #3. If the first argument is 1-dimensional and the second argument is 2-dimensional
                dim_list1 = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(1) ]
                dim_list1 = dim_list + dim_list1
                input_tensor1 = get_tensor(min_val,max_val,dim_list1) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])

        elif len(dim_list) == 2:
            if random.random() < 0.5:
                # 2: If both arguments are 2-dimensional, the matrix-matrix product is returned. 
                input_tensor1 = get_tensor(min_val,max_val,dim_list[::-1]) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])
            else:            
                #4. If the first argument is 2-dimensional and the second argument is 1-dimensional
                input_tensor1 = get_tensor(min_val,max_val,[dim_list[-1]]) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])

        
        elif len(dim_list) >= 3:
            randome_choice = random.random()
            if randome_choice < 0.33:
                #5. # batched matrix x broadcasted vector
                dim_list1 = [dim_list[2]]
                input_tensor1 = get_tensor(min_val,max_val,dim_list1) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])
            elif randome_choice >= 0.33 and randome_choice < 0.66: 
                #6. batched matrix x batched matrix
                dim_list1 = [dim_list[0], dim_list[2], random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)]
                input_tensor1 = get_tensor(min_val,max_val,dim_list1) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])
            else:
                #7. batched matrix x broadcasted matrix
                dim_list1 = [dim_list[2], random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)]
                input_tensor1 = get_tensor(min_val,max_val,dim_list1) 
                log_input_output = log_input_output + fuzz_matmul([input_tensor,input_tensor1])

    return(log_input_output)


'''unit_test'''
def test_fuzz_matmul(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_matmul() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_matmul", log_input_output)

if UNIT_TEST is True:      
    test_fuzz_matmul()

In [ ]:
def fuzz_any(input_tensor_list=None):
    
    api_name = 'any'

    tensor_dimension = choice(dimension_choice)
    dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
    log_input_output = []
    min_val = MIN_TENSOR_VALUE
    max_val = MAX_TENSOR_VALUE

    if input_tensor_list is None:
        if random.random() < 0.5:
            input_tensor = torch.rand(tuple(dim_list)) < 0.33
        else:
            input_tensor = (get_tensor(min_val,max_val,dim_list) % 2) * random.randint(1,5)
    else:
        input_tensor = input_tensor_list[0]

    d = random.randint(0,tensor_dimension-1)
    output_tensor = torch.any(input_tensor,dim=d)
    log_input_output.append([api_name,output_tensor,[input_tensor]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_any(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_any() #0,5,choice(dimension_choice))
        print("fuzz_any" , t)

if UNIT_TEST is True:
    test_fuzz_any()

In [ ]:
def fuzz_ne(input_tensor_list=None):

    api_name0 = 'ne'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)

    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    if random.random() < 0.33:
        flatten_input_tensor = torch.flatten(input_tensor)
        scalar_value=flatten_input_tensor[torch.randint(len(flatten_input_tensor),(1,))].item()
        output_tensor = torch.ne(input_tensor, scalar_value) 
        log_input_output.append([api_name0,output_tensor,[input_tensor, torch.tensor(scalar_value)]])
    else:
        input_tensor1 = input_tensor - get_tensor(0, 10,list(input_tensor.shape))
        if random.random() < 0.5:
            input_tensor, input_tensor1 = swap_tensor(input_tensor, input_tensor1)
        output_tensor = torch.ne(input_tensor, input_tensor1) 
        log_input_output.append([api_name0,output_tensor,[input_tensor, input_tensor1]])
    
    return(log_input_output)


'''unit_test'''
def test_fuzz_ne(): 
    for i in range(1,NUM_FUZZ_PER_API): #number of call
        log_input_output = fuzz_ne()#MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_ne", log_input_output)

if UNIT_TEST is True:  
    test_fuzz_ne()

In [ ]:
def fuzz_add(input_tensor_list=None):

    api_name = 'add'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    

    if random.random() > 0.33:
        input_tensor1 = get_broadcast_tensor(input_tensor, MIN_TENSOR_VALUE, MAX_TENSOR_VALUE)
        if random.random() < 0.5:
            input_tensor, input_tensor1 = swap_tensor(input_tensor, input_tensor1)
        output_tensor = torch.add(input_tensor, input_tensor1) 
        log_input_output.append([api_name,output_tensor,[input_tensor, input_tensor1]])
    else:
        scalar_value = random.randint(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE)
        output_tensor = torch.add(input_tensor, scalar_value)
        log_input_output.append([api_name,output_tensor,[input_tensor,torch.tensor(scalar_value)]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_add(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_add() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_add", t)
        
if UNIT_TEST is True:
    test_fuzz_add()

In [ ]:
def fuzz_roll(input_tensor_list=None):

    api_name = 'roll'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    
    size = len(list(input_tensor.shape))
    shift = random.randint(1, MAX_VAL_PER_DIMENSION)
    dim = random.randint(0, size-1)
    output_tensor = torch.roll(input_tensor, shift, dim)

    log_input_output.append([api_name,output_tensor,[input_tensor]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_roll(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_roll() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_roll", t)
        
if UNIT_TEST is True:
    test_fuzz_roll()

In [ ]:
def fuzz_arange(input_tensor_list=None):

    api_name = 'arange'
    log_input_output = []

    if input_tensor_list is None:
        input_tensor = random.randint(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE)
    else:
        input_tensor = input_tensor_list[0]
    
    output_tensor = torch.arange(input_tensor)

    log_input_output.append([api_name,output_tensor,[]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_arange(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_arange() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_arange", t)
        
if UNIT_TEST is True:
    test_fuzz_arange()

In [ ]:
def fuzz_mul(input_tensor_list=None):

    api_name = 'mul'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    

    if random.random() > 0.33:
        input_tensor1 = get_broadcast_tensor(input_tensor, MIN_TENSOR_VALUE, MAX_TENSOR_VALUE)

        if random.random() < 0.5:
            #swap
           input_tensor, input_tensor1 = swap_tensor(input_tensor, input_tensor1)
        
        output_tensor = torch.mul(input_tensor, input_tensor1) 
        log_input_output.append([api_name,output_tensor,[input_tensor, input_tensor1]])

    else:
        scalar_value = random.randint(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE)

        output_tensor = torch.mul(input_tensor, scalar_value)
        
        log_input_output.append([api_name,output_tensor,[input_tensor,torch.tensor(scalar_value)]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_mul(): 
    for i in range(1,NUM_FUZZ_PER_API): #number of call
        t=fuzz_mul() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_mul", t)
        
if UNIT_TEST is True:
    test_fuzz_mul()

In [ ]:
def fuzz_div(input_tensor_list=None):

    api_name = 'div'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    

    if random.random() > 0.5:
        input_tensor1 = get_broadcast_tensor(input_tensor, 1, MAX_TENSOR_VALUE)
        output_tensor = torch.div(input_tensor, input_tensor1) 
        log_input_output.append([api_name,output_tensor,[input_tensor, input_tensor1]])
    else:
        scalar_value = random.randint(1,MAX_TENSOR_VALUE)
        output_tensor = torch.div(input_tensor, scalar_value)
        log_input_output.append([api_name,output_tensor,[input_tensor,torch.tensor(scalar_value)]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_div(): 
    for i in range(1,NUM_FUZZ_PER_API): #number of call
        t=fuzz_div() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_div", t)
        
if UNIT_TEST is True:
    test_fuzz_div()

In [ ]:
'''
 Concatenates a sequence of tensors along a new dimension.
'''

def fuzz_stack(input_tensor_list=None):
   
    api_name = 'stack'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    if random.random() < 0.1:
        input_tensor1 = input_tensor
    else:
        input_tensor1 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,list(input_tensor.shape))

    if random.random() < 0.5:
            #swap
            input_tensor, input_tensor1 = swap_tensor(input_tensor, input_tensor1)

    dimension = len(list(input_tensor1.size())) - 1
    dim = random.randint(0,dimension)

    output_tensor=torch.stack((input_tensor,input_tensor1),dim)  #p = x y cat_dim
    log_input_output.append([api_name,output_tensor,[input_tensor, input_tensor1]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_stack(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_stack()#MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_stack", t)


if UNIT_TEST is True:
    test_fuzz_stack()

In [ ]:
import torch.nn.functional as F

def fuzz_one_hot(input_tensor_list=None):

    api_name = 'one_hot'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    max_val = torch.max(input_tensor)+1
    num_classes = random.randint(max_val+1, max_val+2)
    output_tensor = torch.nn.functional.one_hot(input_tensor, num_classes)
    
    log_input_output.append([api_name,output_tensor,[input_tensor, torch.tensor(num_classes)]])

    return(log_input_output)

    

'''unit_test'''
def test_fuzz_one_hot(): 
    for i in range(1,NUM_FUZZ_PER_API): #number of call
        t=fuzz_one_hot() 
        print("fuzz_one_hot" , t)

if UNIT_TEST is True:
    test_fuzz_one_hot()

In [ ]:
'''
  Returns the indices of the maximum value of all elements in the input tensor.
'''

def fuzz_argmax(input_tensor_list=None):
    api_name = 'argmax'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            input_tensor = input_tensor.float()

    dim = random.randint(0, len(input_tensor.shape))
   
    if dim == len(input_tensor.shape):
        output_tensor = torch.argmax(input_tensor)
    else: 
        output_tensor = torch.argmax(input_tensor,dim=dim)
    
    log_input_output.append([api_name,output_tensor,[input_tensor]])
    return(log_input_output)


'''unit_test'''
def test_fuzz_argmax(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_argmax() 
        print("fuzz_argmax" , t)

if UNIT_TEST is True:
    test_fuzz_argmax()

In [ ]:
'''
Gathers values along an axis specified by dim.
'''
def fuzz_gather(input_tensor_list=None):

    api_name = 'gather'

    log_input_output = []

    
    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])


    if input_tensor_list is not None and len(input_tensor_list) >= 2:
        input_tensor1 = input_tensor_list[1]
    else:
        '''
            input and index must have the same number of dimensions. 
            It is also required that index.size(d) <= input.size(d) for all 
            dimensions d != dim. out will have the same shape as index
        '''
        probability = [True, False, False]
        dim_list = input_tensor.shape
        dim_list1 = []
        for d in dim_list:
            if (random.choice(probability) == True) or (d==1): 
                dim_list1.append(d)
            else:
                dim_list1.append(d-1)
        input_tensor1 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list1) % min(dim_list)

    d = random.randint(0,len(list(input_tensor1.size()))-1)
    output_tensor = torch.gather(input_tensor,d, input_tensor1)
    log_input_output.append([api_name,output_tensor,[input_tensor, input_tensor1]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_gather(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_gather()#(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_gather" , t)


if UNIT_TEST is True:     
    test_fuzz_gather()

In [ ]:
def fuzz_max(input_tensor_list=None):

    api_name = 'max'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    dimension = len(list(input_tensor.size()))
    dim = random.randint(0, dimension)

    if dim == dimension:
        output_tensor = torch.max(input_tensor)
        log_input_output.append([api_name,output_tensor,[input_tensor]])
    else:
        output_tensor = torch.max(input_tensor, dim=dim)
        log_input_output.append([api_name,output_tensor.values,[input_tensor]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_max(): 
    for i in range(1,NUM_FUZZ_PER_API): #number of call
        t=fuzz_max() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_max" , t)

if UNIT_TEST is True:
    test_fuzz_max()

In [ ]:
'''
Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
'''
def fuzz_eye(input_tensor_list=None):

    api_name = 'eye'

    log_input_output = []

    scalar = random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)
    output_tensor = torch.eye(scalar)
    log_input_output.append([api_name,output_tensor,[]])    
    return(log_input_output)

'''unit_test'''
def test_fuzz_eye(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_eye() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_eye" , t)

if UNIT_TEST is True:     
    test_fuzz_eye()

In [ ]:
'''
    Constructs a tensor by repeating the elements of input. 
    The reps argument specifies the number of repetitions in each dimension.
'''
def fuzz_tile(input_tensor_list=None):

    api_name = 'tile'

    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])
    
    tensor_dimension = len(list(input_tensor.size()))
    dims = tuple(random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for _ in range(tensor_dimension+1))
    output_tensor = torch.tile(input_tensor, dims=dims)
    log_input_output.append([api_name,output_tensor,[input_tensor]])

    return(log_input_output)

'''unit_test'''
def test_fuzz_tile(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_tile()
        print("fuzz_tile" , t)

if UNIT_TEST is True:
    test_fuzz_tile()

In [ ]:
def fuzz_sum(input_tensor_list=None):

    api_name = 'sum'
    log_input_output = []

    

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    dimension = len(list(input_tensor.size()))
    dim = random.randint(0, dimension)
    if dim == dimension:
        output_tensor = torch.sum(input_tensor)
    else:
        output_tensor = torch.sum(input_tensor, dim=dim)
    log_input_output.append([api_name,output_tensor,[input_tensor]])

    return(log_input_output)

'''unit_test'''
def test_fuzz_sum(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_sum() 
        print("fuzz_sum" , t)

if UNIT_TEST is True:
    test_fuzz_sum()

In [ ]:
def fuzz_searchsorted(input_tensor_list=None): 
  
    api_name = 'searchsorted'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    
    input_tensor = torch.sort(input_tensor)[0]
    size = list(input_tensor.shape)
    
    if len(size) > 1:
        in2_size = size[:-1] + [random.randint(MIN_VAL_PER_DIMENSION, MAX_VAL_PER_DIMENSION)]
        in2 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,in2_size)   
    else:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        in2 = get_tensor(MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,dim_list)
       
    
    output_tensor = torch.searchsorted(input_tensor, in2)
    log_input_output.append([api_name,output_tensor,[input_tensor, in2]])    
    return(log_input_output)

'''unit_test'''
def test_fuzz_searchsorted(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        log_input_output = fuzz_searchsorted() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_searchsorted", log_input_output)

if UNIT_TEST is True:      
    test_fuzz_searchsorted()

In [ ]:
def fuzz_square(input_tensor_list=None):
    api_name = 'square'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    output_tensor = torch.square(input_tensor)
    log_input_output.append([api_name,output_tensor,[input_tensor]])

    return(log_input_output)


'''unit_test'''
def test_fuzz_square(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_square()
        print("fuzz_square" , t)

if UNIT_TEST is True:
    test_fuzz_square()

In [ ]:
'''
    Constructs a tensor by repeating the elements of input. 
    The reps argument specifies the number of repetitions in each dimension.
'''
def fuzz_repeat_interleave(input_tensor_list=None):

    api_name = 'repeat_interleave'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])


    dim_list = list(input_tensor.size())
    tensor_dimension = len(dim_list)
    d = random.randint(0,tensor_dimension-1)
    repeat_val = random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION)
    output_tensor = torch.repeat_interleave(input_tensor, repeat_val, d)
    log_input_output.append([api_name,output_tensor,[input_tensor]]) 

    return(log_input_output)


'''unit_test'''
def test_fuzz_repeat_interleave(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_repeat_interleave()
        print("fuzz_repeat_interleave" , t)

if UNIT_TEST is True:
    test_fuzz_repeat_interleave()

In [ ]:
def fuzz_cdist(input_tensor_list=None):
    api_name = 'cdist'
    log_input_output = []

    if input_tensor_list is None:
        dim1 = choice(dimension_choice)
        dim2 = choice(dimension_choice)
        dim3 = choice(dimension_choice)
        dim_list = [dim1,dim2,dim3]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if len(list(input_tensor.shape)) != 3:
            return log_input_output
        if input_tensor.dtype == torch.bool:
            return([-1])

    dimension = list(input_tensor.shape)
    
    dim = choice(dimension_choice)

    dim_list1 = [dimension[0],dim,dimension[2]]
    input_tensor1 = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE),dim_list1)

    p = choice([0,1,2])
    output_tensor = torch.cdist(input_tensor.float(),input_tensor1.float(),p=p)
    log_input_output.append([api_name,output_tensor,[input_tensor, input_tensor1]])  

    return(log_input_output)


'''unit_test'''
def test_fuzz_cdist(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_cdist() #MIN_TENSOR_VALUE,MAX_TENSOR_VALUE,choice(dimension_choice))
        print("fuzz_cdist" , t)

if UNIT_TEST is True:
    test_fuzz_cdist()

In [ ]:
import numpy as np
import itertools

# Generativing sets of divisors for 0-125 values. fuzz_reshape uses this divisor.
divisors = {}
divisors[1] = []
for target in range(2, 125+1):
    divisors[target] = []
    for i in range(2, target+1):
        if target % i == 0:
            cur_divisors = [i]
            if len(divisors[target/i]) == 0:
                divisors[target].append(cur_divisors)
            else:
                for sub in divisors[target/i]:
                    divisors[target].append(cur_divisors + sub)

def constrainedRandoms(ranges, constrain):
    answer = []
    for vector in itertools.product(*ranges):
        print('vector', vector)
        if np.prod(vector) == constrain:
            answer.append(vector)
    return answer

'''
Returns a tensor with the same data and number of elements as input, but with the specified shape. 
'''
def fuzz_reshape(input_tensor_list=None):

    api_name = 'reshape'
    log_input_output = []

    if input_tensor_list is None:
        tensor_dimension=choice(dimension_choice)
        dim_list = [  random.randint(MIN_VAL_PER_DIMENSION,MAX_VAL_PER_DIMENSION) for x in range(tensor_dimension) ]
        input_tensor = get_tensor(MIN_TENSOR_VALUE,int(MAX_TENSOR_VALUE/2),dim_list)
    else:
        input_tensor = input_tensor_list[0]
        if input_tensor.dtype == torch.bool:
            return([-1])

    in1_elements = torch.numel(input_tensor)
    if in1_elements > f.EMBEDDING_SIZE or in1_elements == 1:
        return []
    
    shape = list(random.choice(divisors[in1_elements]))
    if random.random() < 0.25:
        shape.insert(random.randint(0, len(shape)), 1)
    if random.random() < 0.25:
        shape[random.randint(0, len(shape)-1)] = -1
    if len(shape) > MAX_VAL_PER_DIMENSION:
        return []
    
    output_tensor = torch.reshape(input_tensor, tuple(shape))

    log_input_output.append([api_name,output_tensor,[input_tensor]])      

    return(log_input_output)

'''unit_test'''
def test_fuzz_reshape(): 
    for i in range(1,NUM_TEST_FUZZ): #number of call
        t=fuzz_reshape()
        print("fuzz_reshape" , t)

if UNIT_TEST is True:
    test_fuzz_reshape()

Fuzzing Calls

In [ ]:
api_seq_list = [
         ['expand', 'transpose'], #SO1
         ['lt', 'where'],  #SO2.1
         ['minimum'], #SO2.2
         ['tensordot'],  #SO5, 
         ['unsqueeze','eq'],  #SO6
         ['transpose'], #SO7
         ['gt', 'masked_select'],  #SO8
         ['bincount'],   #SO11
         ['matmul', 'transpose'],  #SO13
         ['any'],             #SO14
         ['ne', 'mul'],       #SO15
         ['unsqueeze', 'mul'], #SO16
         ['stack'],  #SO17
         ['matmul', 'add'],   #SO18
         ['argmax', 'one_hot'],   #SO20
         ['gather'],               #SO21
         ['tensordot'],  #SO22
         ['unsqueeze', 'matmul', 'squeeze'], #SO22.1
         ['one_hot', 'max'],  #SO23.1
         ['one_hot', 'sum'],  #SO23.2
         ['div', 'where'],   #SO24.1
         ['ne','div', 'where'], #SO24.2
         ['eye', 'tile'],  #SO25
         ['sum', 'sum'],  #SO26
         ['one_hot','max'], #SO27
         ['searchsorted'], #SO29
         ['cdist'],   #SO30
         ['arange', 'tensordot'], #SO32
         ['tensordot'],  #SO34
         ['add', 'div'],  #SO36.1
         ['arange','add','div'], #SO36.2
         ['tensordot'],  #SO37
         ['mul', 'square'],  #39
         ['arange', 'ne', 'masked_select'], #SO41
         ['ne', 'masked_select'],  #SO41.1
         ['eq'],   #SO42
         ['reshape','sum'], #SO44
         ['unsqueeze', 'roll', 'where'],  #SO45.1
         ['roll', 'unsqueeze', 'where'],  #SO45.2
         ['arange', 'repeat_interleave'],  #SO46
         ['unsqueeze', 'eq', 'argmax'],  #SO48
         ['unsqueeze', 'eq', 'arange'],  #SO48.2
         ['transpose', 'mul', 'transpose'], #SO49
         ['expand','one_hot']  #SO50
    ]

In [ ]:
#https://pytorch.org/docs/stable/torch.html
api_dict = {
    #Creation Ops
    'arange':fuzz_arange,
    'eye':fuzz_eye,

    #Indexing, Slicing, Joining, Mutating Ops
    'gather':fuzz_gather,
    'masked_select':fuzz_masked_select,
    'reshape':fuzz_reshape,
    'squeeze':fuzz_squeeze,
    
    'stack':fuzz_stack,
    'tile':fuzz_tile,
    'transpose':fuzz_transpose,
    'unsqueeze':fuzz_unsqueeze,
    'where':fuzz_where,
    

    #Math operations 
    'add':fuzz_add,
    'div': fuzz_div,
    'mul':fuzz_mul,
    'matmul':fuzz_matmul,
    'square':fuzz_square,
    

    #Reduction Ops
     'argmax':fuzz_argmax,
     'any':fuzz_any,
     'sum':fuzz_sum,

    #Comparison Ops
    'eq':fuzz_eq,
    'ne':fuzz_ne,
    'gt':fuzz_gt,
    'lt':fuzz_lt,        
    'max':fuzz_max,
    'minimum':fuzz_minimum,

    #Other
    'bincount':fuzz_bincount,
    'cdist':fuzz_cdist,
    'expand':fuzz_expand,
    'repeat_interleave': fuzz_repeat_interleave,
    'roll':fuzz_roll,
    'searchsorted':fuzz_searchsorted,
    'tensordot':fuzz_tensordot,

    #torch.nn.functional module
     'one_hot':fuzz_one_hot,
}


In [ ]:
def Fuzz_Single_API(api_seq):
    global api_dict
    logs = []

    assert(len(api_seq) == 1)

    api0 = api_seq[0]
                        
    fuzzing_method0 = api_dict[api0]
    domain_io0 = fuzzing_method0()

    for io0 in domain_io0:
        logs.append([io0])

    return(logs)

'''Unit Test'''
def Test_Fuzz_Single_API():
   apis = ['tensordot']
   logs = Fuzz_Single_API(apis)
   for l in logs:
       print(l)

Test_Fuzz_Single_API()

In [ ]:
def Fuzz_Two_APIs(api_seq):
    global api_dict
    global api_dict_tensor
    logs = []

    assert(len(api_seq) == 2)
        
    api0, api1 = api_seq[:]
    
    fuzzing_method0 = api_dict[api0]
    fuzzing_method1 = api_dict[api1]

    domain_io0 = fuzzing_method0()
    
    for io in domain_io0:
        input1 = io[1]
        try:
            domain_io1 = fuzzing_method1([input1])
            
            if len(domain_io1) == 0:
                continue
            for io1 in domain_io1:
                io_seq = [io, io1]
                logs.append(io_seq)
        except:
            continue

    return(logs)

'''Unit Test'''
def Test_Fuzz_Two_APIs():
   apis =  ['unsqueeze', 'reshape'] #['add', 'div'] # ['lt', 'where']
   logs = Fuzz_Two_APIs(apis)
   for l in logs:
       print(l)

Test_Fuzz_Two_APIs()

In [ ]:
import sys
def Fuzz_Three_APIs(api_seq):
    global api_dict
    global api_dict_tensor
    logs = []

    assert(len(api_seq) == 3)
        
    api0, api1, api2 = api_seq[:]
    
    fuzzing_method0 = api_dict[api0]
    fuzzing_method1 = api_dict[api1]
    
    fuzzing_method2 = api_dict[api2]

    domain_io0 = fuzzing_method0()

    for io0 in domain_io0:
        input1 = io0[1]
        try:
            domain_io1 = fuzzing_method1([input1])
            
            for io1 in domain_io1:
                input2 = io1[1]
                
                try:
                    domain_io2 = fuzzing_method2([input2])
                    for io2 in domain_io2:
                        io_seq = [io0, io1, io2]
                        logs.append(io_seq)
                except:
                    continue
        except:
            continue

    return(logs)

'''Unit Test'''
def Test_Fuzz_Three_APIs():
   apis = ['arange','add','div'] #['transpose', 'mul', 'transpose']# ['unsqueeze', 'eq', 'argmax'] # ['unsqueeze', 'matmul', 'squeeze']
   logs = Fuzz_Three_APIs(apis)
   for l in logs:
       print(l)

Test_Fuzz_Three_APIs()

In [ ]:
''' RQ2a: Large Given Sequence'''
import random 
import itertools 

def get_random_pairs(org_list, seq_number): 
  pairs = [p for p in itertools.product(org_list, repeat=seq_number)]
  return pairs 

n = ['a','b','c']

api_seq_set = set()
api_seql = []
api_set = set()

for api_seq in api_seq_list:
    if len(api_seq) > 1:
      api_seql.append(api_seq)
      for api in api_seq:
          api_set.add(api)
    else:
      api_set.add(api_seq[0])
api_list = ([[el] for el in list(api_set)]) + api_seql
# print(api_list, len(api_list), len(list(api_set)))

api_seq_list = api_list


In [ ]:
''' RQ2b: Related Sequence'''
import random 
import itertools 

def unique(list1):
    x = np.array(list1)
    u = np.unique(x)
    return(list(u))
    

def get_random_pairs(org_list, seq_number): 
  pairs = [p for p in itertools.permutations(org_list, r=seq_number)]
  return pairs 


n = ['a','b','c']

api_seq_set = set()
api_seql = []
api_set = set()

for api_seq in api_seq_list:
    if len(api_seq) > 1:
      for api in api_seq:
          api_set.add(api)
      api_alt_seq = []
      api_seq_len = len(api_seq)
      
      api_alt_seq = get_random_pairs(api_seq,api_seq_len)
      api_alt_seq.remove(tuple(api_seq))
      api_alt_seq = [list(el) for el in api_alt_seq]
      api_seql += api_alt_seq
      
api_list = ([[el] for el in list(api_set)]) 

api_list += unique(api_seql)
print(api_list, len(api_list), len(list(api_set)))

In [ ]:
# 
''' RQ2b': Random 2 seq'''
import random 
import itertools 

def get_random_pairs(org_list, seq_number): 
  pairs = [p for p in itertools.permutations(org_list, r=seq_number)]
  return pairs 

n = ['a','b','c']

api_seq_set1 = []
api_seql1 = []
api_set1 = set()

for api_seq in api_seq_list:
    if len(api_seq) == 2:
      api_seq_set1.append(tuple(api_seq))
      for api in api_seq:
          api_set1.add(api)
    
api_comb = get_random_pairs(list(api_set1),2)
print(len(api_comb))
print(len(api_seq_set1))

#remove the pairs already present
for s in api_seq_set1:
  if s in api_comb:
    api_comb.remove(s)
print(len(api_comb))


list_of_random_items = random.sample(api_comb, 300)
list_of_random_items = ([list(el) for el in list(list_of_random_items)]) 

print(len(list_of_random_items))
print(list_of_random_items)


In [ ]:
''' RQ2b'': Random 3 seq'''
import random 
import itertools 

def get_random_pairs(org_list, seq_number): 
  pairs = [p for p in itertools.permutations(org_list, r=seq_number)]
  return pairs 

n = ['a','b','c']

api_seq_set1 = []

api_seql1 = []
api_set1 = set()

for api_seq in api_seq_list:
      for api in api_seq:
          api_set1.add(api)
    
api_comb = get_random_pairs(list(api_set1),3)

#remove the pairs already present
for s in api_seq_set1:
  if s in api_comb:
    api_comb.remove(s)

list_of_random_items = random.sample(api_comb, 30)
list_of_random_items = ([list(el) for el in list(list_of_random_items)]) 


In [ ]:
def filter_comb(comb):

    api_comb_filtered = comb.copy()
    count = 0

    for api_seq in comb: #api_seq_list:
        l = []
        if len(api_seq) == 1:
            l = Fuzz_Single_API(api_seq)
        elif len(api_seq) == 2:
            l = Fuzz_Two_APIs(api_seq)
            
            for i in l:
                if i[-1] == -1:
                    count += 1
                    print(api_seq , '\n')
                    api_comb_filtered.remove(api_seq)
        elif len(api_seq) == 3:
            l = Fuzz_Three_APIs(api_seq)
            for i in l:
                if i[-1] == -1:
                    count += 1
                    print(api_seq , '\n')
                    api_comb_filtered.remove(api_seq)
                    
    print(count)
    return(api_comb_filtered)

api_comb_filtered = filter_comb(api_list)

In [ ]:
SAVE_FILE = f.fuzzing

print("saving output at %s" % SAVE_FILE)

def Fuzz_Core_APIs():
    global api_dict
    global api_list
    global api_seq_list
    logs = []
    count = 0

    api_list = list(api_dict.keys())   
    print(len(api_list)) 

    for i in range(1, f.NUM_FUZZ_PER_API):
        if (i%1000 == 0):
            print('---------------- Fuzzing %d times ----------------' % i)
        if (i%10000 == 0):
            print('---------------- Fuzzing %d times ----------------' % i)
            fuzz_file = f.Path + '/fuzzing_data/' + str(i) + '_' + SAVE_FILE
            torch.save(logs,f=fuzz_file)
            logs.clear()


        if f.COMPOSITE is False:

            for api_name, fuzz_method in api_dict.items():

                l = Fuzz_Single_API([api_name])
                print(api_name, len(l))
                logs += l
        else:
            for api_seq in api_comb_filtered: #final_gen_test: #api_comb_filtered: #api_seq_list:
                l = []
                if len(api_seq) == 1:
                    l = Fuzz_Single_API(api_seq)
                elif len(api_seq) == 2:
                    l = Fuzz_Two_APIs(api_seq)
                elif len(api_seq) == 3:
                    l = Fuzz_Three_APIs(api_seq)
                else:
                    continue
                logs += l
          

    return(logs)


In [ ]:
Fuzz_Core_APIs()